In [48]:
import requests
from datetime import datetime as dt

def parse_data(url):
    resp = requests.get(url).json()
    data = []
    for indx, ts in enumerate(resp['chart']['result'][0]['timestamp']):
        data.append((dt.fromtimestamp(int(ts)).strftime('%Y%m%d%H%M%S'), 
                     resp['chart']['result'][0]['indicators']['quote'][0]['open'][indx],
                     resp['chart']['result'][0]['indicators']['quote'][0]['high'][indx],
                     resp['chart']['result'][0]['indicators']['quote'][0]['close'][indx],
                     resp['chart']['result'][0]['indicators']['quote'][0]['low'][indx],
                     resp['chart']['result'][0]['indicators']['quote'][0]['volume'][indx]
                    ))
    return data

def save_to_file(data_save, file_name):
    """save data to filename"""
    with open("{}.txt".format(file_name), "a") as f:
        f.write(('timestamp, open, high, close, low, volume'))
        f.write('\n')
        for i in data_save:
            f.write(str(i))
            f.write('\n')     

            
if __name__ == '__main__':
    url = 'https://query1.finance.yahoo.com/v7/finance/chart/%5EBSESN?period2=	1478111136&period1=1478012261&interval=1m&indicators=quote&includeTimestamps=true&includePrePost=true&events=div%7Csplit%7Cearn&corsDomain=finance.yahoo.com'

    url2 = 'https://query1.finance.yahoo.com/v7/finance/chart/%5ENSEI?period2=	1478111136&period1=1478012261&interval=1m&indicators=quote&includeTimestamps=true&includePrePost=true&events=div%7Csplit%7Cearn&corsDomain=finance.yahoo.com'
    
    data = parse_data(url2)
    save_to_file(data, 'ZINC')

In [94]:
import os
import argparse
from datetime import datetime as dt

def parse_args():
    parser = argparse.ArgumentParser(description="Download data from Yahoo")
    parser.add_argument("symbol", type=str, help="Bloomberg symbol = %5ENSEI")
    parser.add_argument("start", type=int, help="Start Date = 20161029035900 as %Y%m%d%H%M%S")
    parser.add_argument("end", type=int, help="End Date = 20161029035900 as %Y%m%d%H%M%S")
    parser.add_argument("scale", type=int, help="Scale = 100")
    
    args = parser.parse_args()
    return args

def parse_data(symbol, start, end):
    url = 'https://query1.finance.yahoo.com/v7/finance/chart/{0}?period2={1}&period1={2}&interval=1m&indicators=quote&includeTimestamps=true&includePrePost=true&events=div%7Csplit%7Cearn&corsDomain=finance.yahoo.com'.format(symbol, start, end)
    
    resp = requests.get(url).json()
    data = []
    
    for indx, ts in enumerate(resp['chart']['result'][0]['timestamp']):
        data.append((ts, resp['chart']['result'][0]['indicators']['quote'][0]['open'][indx]))
        data.append((ts, resp['chart']['result'][0]['indicators']['quote'][0]['high'][indx]))
        data.append((ts, resp['chart']['result'][0]['indicators']['quote'][0]['low'][indx]))
        data.append((ts, resp['chart']['result'][0]['indicators']['quote'][0]['close'][indx]))
        
    return data

def save_to_file(data, file_name, scale):
    """save data to filename"""
    with open("{}.txt".format(file_name), "a") as f:
        for i in data:
            _time = dt.fromtimestamp(int(i[0]) - 32400).strftime('%y%m%d %H%M%S')
            quote = str(int(round(i[1], 2) * scale))
            f.write(_time + ' ' + quote + '\n')

if __name__ == "__main__":
    symbol_args = parse_args()
    _start = int(dt.strptime(str(symbol_args.start), "%Y%m%d%H%M%S").timestamp())
    _end = int(dt.strptime(str(symbol_args.end), "%Y%m%d%H%M%S").timestamp())
    
    data = parse_data(symbol_args.symbol, _start, _end)
    save_to_file(data, symbol_args.symbol, symbol_args.scale)

usage: __main__.py [-h] symbol start end scale
__main__.py: error: the following arguments are required: start, end, scale


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.
